In [1]:
from helper import *
from Simulation.WaitTime import WaitTime
from Simulation.MoveJoint import MoveJoint
from Simulation.MoveLinear import MoveLinear
from Simulation.MoveTrajectory import MoveTrajectory
from Simulation.Simulation import Simulation
from neura_dual_quaternions import DualQuaternion, Quaternion

from DualQuaternionQuinticBlends.LineGenerator import LineGenerator
from DualQuaternionQuinticBlends.ArcGenerator import ArcGenerator
from DualQuaternionQuinticBlends.DQQBTrajectoryGenerator import DQQBTrajectoryGenerator

In [2]:
q2 = np.array([0, deg2rad(-60), 0, deg2rad(-60), 0, deg2rad(-60), 0])
q3 = -q2

q4 = np.array([deg2rad(0), deg2rad(0), deg2rad(0), deg2rad(90), deg2rad(0), deg2rad(-90), deg2rad(0)])
q5 = -q4

x1 = DualQuaternion.fromQuatPos(Quaternion.fromAxisAngle(np.pi*0.5, np.array([0,1,0])), np.array([0.815,0,1.138]))
x2 = DualQuaternion.fromQuatPos(Quaternion.fromAxisAngle(np.pi*0.5, np.array([0,1,0])), np.array([0.815,0.2,1.138]))     
x3 = DualQuaternion.fromQuatPos(Quaternion.fromAxisAngle(np.pi*0.5, np.array([0,1,0])), np.array([1, 0.3, 0.8]))   

# define orientation quaternions for test trajectory
quat1 = Quaternion(1,0,0,0)
quat2 = Quaternion(0,1,0,0)


# define dual quaternions for linear interpolation
dq1 = DualQuaternion.fromQuatPos(quat1, np.array([1, 0, 0.5]))
dq2 = DualQuaternion.fromQuatPos(quat1, np.array([0.6, 0, 0.5 ]))
dq3 = DualQuaternion.fromQuatPos(quat2, np.array([0.6, 0, 0.5 ]))
dq4 = DualQuaternion.fromQuatPos(quat2, np.array([0.6, 0.3, 0.5 ]))

# assign dual quaternions to lists
line_DQ_list = [dq1, dq2, dq3, dq4]

# initialize line and arc generators
line_generator = LineGenerator()

# define the acceleration and jerk limits
a_cart_max = 2
j_cart_max = 30
a_ang_max = 3
j_ang_max = 30

# define desired cartesian velocites and maximum angular velocity
des_cart_vel_line = 0.1
max_ang_vel = 1

# generate segments
segments = line_generator.generateSegments(line_DQ_list, des_cart_vel_line, max_ang_vel)

# generate trajectory object with the segments defined above
trajectory1 = DQQBTrajectoryGenerator()
trajectory1.generateDynamicTrajectory(segments, a_cart_max, j_cart_max, a_ang_max, j_ang_max)

task_list = np.array([WaitTime(2), MoveJoint(q4, 3), MoveLinear(x1, 2), MoveLinear(dq1, 2), MoveTrajectory(trajectory1)])
task_list = np.array([WaitTime(2), MoveJoint(q4, 3), MoveLinear(x1, 2), MoveLinear(x2, 2), MoveLinear(x3, 2)])
sim = Simulation(task_list)

iteration:  1
acceleration limit violated!
acceleration limit violated!
acceleration limit violated!
acceleration limit violated!
iteration:  2
iteration:  3
iteration:  4
iteration:  5
iteration:  6
iteration:  7
iteration:  8
iteration:  9
iteration:  10
[0.1414213562373135, 0.5, 0.5, 0.14142135623731455]


In [3]:
sim.start()

Done!
